# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [27]:
# testing the conda environment, this is not related to the project
import sys
print(sys.executable)  

/opt/miniconda3/envs/rnn/bin/python


In [3]:
# imports
import pandas as pd
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [5]:
def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and concatenate into one resultant dataframe.
    """
    df_eng = pd.read_csv("data/movie_reviews_en.csv")
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    
    df_eng['Original Language'] = 'en'
    df_fr['Original Language'] = 'fr'
    df_sp['Original Language'] = 'sp'
    
    column_mapping = {
        'movie_title': 'Title',
        'release_year': 'Year',
        'movie_synopsis': 'Synopsis',
        'movie_review': 'Review'
    }
    
    df_eng = df_eng.rename(columns=column_mapping)
    df_fr = df_fr.rename(columns=column_mapping)
    df_sp = df_sp.rename(columns=column_mapping)
    
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    return df

df = preprocess_data()

In [6]:
df.sample(10)

,Movie / TV Series,Year,Synopsis,Review,Original Language,Titre,Année,Critiques,Título,Año,Sinopsis,Críticas
19,NaN,NaN,"Dans un futur lointain, un mercenaire doit esc...",NaN,fr,Babylon A.D.,2008.0,"""Ce film est un gâchis complet. Les personnage...",NaN,NaN,NaN,NaN
16,NaN,NaN,Deux employés de bureau incompétents se retrou...,NaN,fr,La Tour Montparnasse Infernale,2001.0,"""Je ne peux pas croire que j'ai perdu du temps...",NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,sp,NaN,NaN,NaN,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido..."
20,NaN,NaN,NaN,NaN,sp,NaN,NaN,NaN,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu..."
4,Inception,2010.0,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,Le film suit l'histoire d'un groupe d'amis ric...,NaN,fr,Le Dîner de Cons,1998.0,"""Je n'ai pas aimé ce film du tout. Le concept ...",NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,sp,NaN,NaN,NaN,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s..."
23,NaN,NaN,NaN,NaN,sp,NaN,NaN,NaN,El Laberinto del Fauno,2006,"Durante la posguerra española, Ofelia (Ivana B...","""El Laberinto del Fauno es una película fascin..."
10,NaN,NaN,Cette comédie musicale raconte l'histoire d'un...,NaN,fr,La La Land,2016.0,"""La La Land est un film absolument magnifique ...",NaN,NaN,NaN,NaN
3,The Godfather,1972.0,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [7]:
from transformers import MarianMTModel, MarianTokenizer

fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"

fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

def translate(text: str, model, tokenizer) -> str:
    """
    Function to translate text using a model and tokenizer
    
    Parameters:
        text (str): Single text string to translate
        model: The translation model
        tokenizer: The tokenizer for the model
    Returns:
        str: Translated text
    """
    if not isinstance(text, str):
        text = str(text)
        
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(**inputs)
    translated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return translated


/opt/miniconda3/envs/rnn/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [8]:
fr_mask = df['Original Language'] == 'fr'
sp_mask = df['Original Language'] == 'sp'

#  previous version that I have changed 
'''
for idx in df[fr_mask].index:
    df.at[idx, 'Review'] = translate(df.at[idx, 'Review'], fr_en_model, fr_en_tokenizer)
    df.at[idx, 'Synopsis'] = translate(df.at[idx, 'Synopsis'], fr_en_model, fr_en_tokenizer)

for idx in df[sp_mask].index:
    # Translate review
    df.at[idx, 'Review'] = translate(df.at[idx, 'Review'], es_en_model, es_en_tokenizer)
    # Translate synopsis
    df.at[idx, 'Synopsis'] = translate(df.at[idx, 'Synopsis'], es_en_model, es_en_tokenizer)
'''

translation_configs = {
    'fr': (fr_mask, fr_en_model, fr_en_tokenizer),
    'sp': (sp_mask, es_en_model, es_en_tokenizer)
}

for lang, (mask, model, tokenizer) in translation_configs.items():
    for col in ['Review', 'Synopsis']:
        df.loc[mask, col] = df.loc[mask, col].apply(
            lambda x: translate(x, model, tokenizer)
        )

In [20]:
df.sample(10)

,Movie / TV Series,Year,Synopsis,Review,Original Language,Titre,Année,Critiques,Título,Año,Sinopsis,Críticas,Review Sentiment
7,The Nice Guys,2016.0,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
9,The Island,2005.0,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
5,Blade Runner 2049,2017.0,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
1,The Dark Knight,2008.0,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive
6,Scott Pilgrim vs. the World,2010.0,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative
3,The Godfather,1972.0,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive
4,Inception,2010.0,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive
0,The Shawshank Redemption,1994.0,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive
2,Forrest Gump,1994.0,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive
8,Solo: A Star Wars Story,2018.0,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Negative


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [9]:
import torch

# MPS for Apple machine 
device = (
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline(
    "sentiment-analysis", 
    model=model_name,
    device=device 
)

def analyze_sentiment(text, classifier):
    """
    Function to perform sentiment analysis on text
    """
    result = classifier(text)[0]
    # Return "Positive" if POSITIVE label and score > 0.5, else "Negative"
    return "Positive" if result['label'] == 'POSITIVE' and result['score'] > 0.5 else "Negative"


In [23]:
def clean_text(text):
    if pd.isna(text) or text == 'nan':
        return ""
    return str(text)

df['Review'] = df['Review'].apply(clean_text)

df = df[df['Review'] != ""]

df.loc[:, 'Review Sentiment'] = df['Review'].apply(lambda x: analyze_sentiment(x, sentiment_classifier))

df_corrected = df[['Movie / TV Series', 'Year', 'Synopsis', 'Review', 'Review Sentiment', 'Original Language']]
df_corrected.columns = ['Title', 'Year', 'Synopsis', 'Review', 'Review Sentiment', 'Original Language']

df_corrected = df_corrected.head(30)

output_path = "result/reviews_with_sentiment.csv"
df_corrected.to_csv(output_path, index=False)



In [24]:
df_corrected.sample(10)

,Title,Year,Synopsis,Review,Review Sentiment,Original Language
3,The Godfather,1972.0,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",Positive,en
8,Solo: A Star Wars Story,2018.0,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",Negative,en
0,The Shawshank Redemption,1994.0,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",Positive,en
5,Blade Runner 2049,2017.0,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",Negative,en
7,The Nice Guys,2016.0,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",Negative,en
4,Inception,2010.0,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",Positive,en
2,Forrest Gump,1994.0,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",Positive,en
9,The Island,2005.0,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",Negative,en
1,The Dark Knight,2008.0,Batman (Christian Bale) teams up with District...,"""The Dark Knight is a thrilling and intense su...",Positive,en
6,Scott Pilgrim vs. the World,2010.0,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",Negative,en


In [26]:
df_corrected.to_csv("result/reviews_with_sentiment.csv", index=False)